In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv
import random
import pickle
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from sklearn.metrics import classification_report

In [2]:
# Kaggle dataset directories
train_data_dir = "train"
test_data_dir = "test"

img_size = 48
categories = ['REAL', 'FAKE']

def load_data(data_dir, categories, img_size):
    data = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv.imread(os.path.join(path, img))
                resized_array = cv.resize(img_array, (img_size, img_size))
                resized_array = resized_array / 255.0
                data.append([resized_array, class_num])
            except Exception as e:
                pass
    return data

In [3]:
# Load and preprocess the data
training_data = load_data(train_data_dir, categories, img_size)
testing_data = load_data(test_data_dir, categories, img_size)

random.shuffle(training_data)
random.shuffle(testing_data)

X_train = []
y_train = []
X_test = []
y_test = []

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)

X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [4]:
# Creating the model
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
# Model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [6]:
# Training the model
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

MemoryError: Unable to allocate 5.15 GiB for an array with shape (100000, 48, 48, 3) and data type float64

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

# Predictions
y_pred = model.predict(X_test)
y_predicted = (y_pred > 0.5).astype("int32").flatten()

# Classification report
print(classification_report(y_test, y_predicted))

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Train and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Train and Validation Loss')

plt.show()

In [ ]:
# 儲存模型
model.save("AIGeneratedModel.h5")

In [ ]:
import shutil

# 移動模型檔案到輸出目錄
shutil.move("AIGeneratedModel.h5", "/kaggle/working/AIGeneratedModel.h5")

In [ ]:
# 加載模型以驗證
model_new = tf.keras.models.load_model("AIGeneratedModel.h5")
print("Model loaded successfully")
